In [ ]:
import os
import pandas as pd
from scipy.spatial.distance import cdist
import numpy as np

def assign_roles(df):
    df['frame'] += 1  # フレーム番号を1始まりに変更
    initial_frame = df[df['frame'] == 1].copy()
    
    # すべてのペアの距離を計算
    coordinates = initial_frame[['x', 'y']].values
    dist_matrix = cdist(coordinates, coordinates, metric='euclidean')
    np.fill_diagonal(dist_matrix, np.inf)  # 自分自身との距離を除外
    
    # 近いペアを3組作成
    pairs = []
    used = set()
    for _ in range(3):
        idx1, idx2 = divmod(np.argmin(dist_matrix), len(dist_matrix))
        pairs.append((initial_frame.iloc[idx1], initial_frame.iloc[idx2]))
        used.update([idx1, idx2])
        dist_matrix[idx1, :] = np.inf
        dist_matrix[:, idx1] = np.inf
        dist_matrix[idx2, :] = np.inf
        dist_matrix[:, idx2] = np.inf
    
    # (0, 752.5)との距離でO, Dを決定
    reference_point = np.array([0, 752.5])
    pair_roles = []
    for p1, p2 in pairs:
        d1 = np.linalg.norm(p1[['x', 'y']].values - reference_point)
        d2 = np.linalg.norm(p2[['x', 'y']].values - reference_point)
        if d1 < d2:
            pair_roles.append((p1, "D", p2, "O"))
        else:
            pair_roles.append((p2, "D", p1, "O"))
    
    # y座標の中点で位置を決定
    mid_y_positions = sorted([( (p1['y'] + p2['y']) / 2, p1, role_D, p2, role_O) for (p1, role_D, p2, role_O) in pair_roles], key=lambda x: x[0], reverse=True)
    positions = ["right", "top", "left"]

    role_map = {}
    for i, (mid_y, p1, role_D, p2, role_O) in enumerate(mid_y_positions):
        pos = positions[i]
        role_map[p1['ID']] = f"{role_D}_{pos}"
        role_map[p2['ID']] = f"{role_O}_{pos}"
    
    df['assigned_role'] = df['ID'].map(role_map).fillna("nan_nan")
    return df

input_dir = "../../BoT-SORT_outputs/Indoor/transformed/merged"
output_dir = "../../BoT-SORT_outputs/Indoor/transformed/merged_with_attributes"
os.makedirs(output_dir, exist_ok=True)

for file_name in os.listdir(input_dir):
    if file_name.endswith(".txt"):
        input_path = os.path.join(input_dir, file_name)
        df = pd.read_csv(input_path, header=None, names=["frame", "ID", "x", "y"])
        result_df = assign_roles(df)
        
        output_path = os.path.join(output_dir, file_name)
        result_df.to_csv(output_path, index=False, header=False)
        print(f"Processed file saved to {output_path}")



Processed file saved to filtered_BoT-SORT_outputs/transformed_indoor_merged_with_roles/basket_S6T3_post.txt
Processed file saved to filtered_BoT-SORT_outputs/transformed_indoor_merged_with_roles/basket_S6T4_post.txt
Processed file saved to filtered_BoT-SORT_outputs/transformed_indoor_merged_with_roles/basket_S6T5_post.txt
Processed file saved to filtered_BoT-SORT_outputs/transformed_indoor_merged_with_roles/basket_S6T6_post.txt
Processed file saved to filtered_BoT-SORT_outputs/transformed_indoor_merged_with_roles/basket_S6T7_post.txt
Processed file saved to filtered_BoT-SORT_outputs/transformed_indoor_merged_with_roles/basket_S1T1_pre.txt
Processed file saved to filtered_BoT-SORT_outputs/transformed_indoor_merged_with_roles/basket_S1T2_pre.txt
Processed file saved to filtered_BoT-SORT_outputs/transformed_indoor_merged_with_roles/basket_S1T3_pre.txt
Processed file saved to filtered_BoT-SORT_outputs/transformed_indoor_merged_with_roles/basket_S1T4_pre.txt
Processed file saved to filtered